In [1]:
import pandas as pd
ks=pd.read_csv('G:/MINE/SUB/Thesis/kaggle/6.Feature Engineering/DataSet/ks-projects-201801.csv',parse_dates=['deadline','launched'])
ks=ks.query("state!='live'")
ks=ks.assign(outcome=(ks.state=='successful').astype(int))
ks=ks.assign(hour=ks.launched.dt.hour,
            day=ks.launched.dt.day,
            month=ks.launched.dt.month,
            year= ks.launched.dt.year)
num_col=['goal','hour','day','month','year','outcome']
cat_col=['category','currency','country']
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
encode=ks[cat_col].apply(encoder.fit_transform)
baseline=ks[num_col].join(encode)
import itertools
interaction=pd.DataFrame(index=ks.index)
for col1,col2 in itertools.combinations(cat_col,2):
    col_name=col1+'_'+col2
    col_value=ks[col1].map(str)+'_'+ks[col2].map(str)
    label_encoder=LabelEncoder()
    interaction[col_name]=label_encoder.fit_transform(col_value)

interBaseline=baseline.join(interaction)
interBaseline.head()
release=pd.Series(ks.index,index=ks.launched,name='count_7days').sort_index()
count_7days=release.rolling('7d').count()-1
count_7days.index=release.values
count_7days=count_7days.reindex(ks.index)
cout_inter_baseline=interBaseline.join(count_7days)
sameProject=pd.DataFrame(ks[['category','launched']],index=ks.index).sort_values('launched')
def time_difference(series):
    return series.diff().dt.total_seconds()/3600
timedeltas=sameProject.groupby('category').transform(time_difference)
timedeltas=timedeltas.fillna(timedeltas.max())
final_baseline=cout_inter_baseline.join(timedeltas.rename({'launched':'time_since_last_Project'},axis=1))
def make_split(df,fraction=.1):
    size=int(len(df)*fraction)
    train=df[:-2*size]
    valid=df[-2*size:-size]
    test=df[-size:]
    return train,valid,test

def train_model(train,valid,test=None,feature=None):
    import lightgbm as lgb
    from sklearn.metrics import roc_auc_score
    if feature is None:
        feature=train.columns.drop('outcome')
    dtrain=lgb.Dataset(train[feature],label=train['outcome'])
    dvalid=lgb.Dataset(valid[feature],label=valid['outcome'])
    param={'num_leaves':64,'objective':'binary','metric':'auc','seed':7}
    num_round=1000
    model=lgb.train(param,dtrain,num_round,valid_sets=[dvalid],verbose_eval=False,early_stopping_rounds=20)
    validPredict=model.predict(valid[feature])
    print('\n'*4)
    print("wait a second to see accuracy on valid sets .. .")
    print("valid accuracy = {}".format(roc_auc_score(valid['outcome'],validPredict)))
    if test is not None:
        print('wait another second to see accuracy on test set .. .')
        print('test accuracy = {}'.format(roc_auc_score(test['outcome'],model.predict(test[feature]))))

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
train,valid,test=make_split(final_baseline)
feature_col=baseline.columns.drop('outcome')
x_l,y_l=train[feature_col],train['outcome']

logistic=LogisticRegression(C=1,penalty='l1',solver='liblinear',random_state=7).fit(x_l,y_l)
model=SelectFromModel(logistic,prefit=True)
x_new=model.transform(x_l)
x_new

array([[1.00e+03, 1.20e+01, 1.10e+01, ..., 1.08e+02, 5.00e+00, 9.00e+00],
       [3.00e+04, 4.00e+00, 2.00e+00, ..., 9.30e+01, 1.30e+01, 2.20e+01],
       [4.50e+04, 0.00e+00, 1.20e+01, ..., 9.30e+01, 1.30e+01, 2.20e+01],
       ...,
       [2.50e+03, 0.00e+00, 3.00e+00, ..., 1.04e+02, 1.30e+01, 2.20e+01],
       [2.60e+03, 2.10e+01, 2.30e+01, ..., 5.80e+01, 1.00e+00, 3.00e+00],
       [2.00e+04, 1.60e+01, 4.00e+00, ..., 5.20e+01, 1.30e+01, 2.20e+01]])

In [25]:
x_new[:1,:]

array([[1000.,   12.,   11.,    8., 2015.,  108.,    5.,    9.]])

In [26]:

selected_feature=pd.DataFrame(model.inverse_transform(x_new),index=train.index,columns=feature_col)
final_col=selected_feature.columns[selected_feature.var()!=0]


In [27]:
selected_feature.head()

,goal,hour,day,month,year,category,currency,country
0,1000.0,12.0,11.0,8.0,2015.0,108.0,5.0,9.0
1,30000.0,4.0,2.0,9.0,2017.0,93.0,13.0,22.0
2,45000.0,0.0,12.0,1.0,2013.0,93.0,13.0,22.0
3,5000.0,3.0,17.0,3.0,2012.0,90.0,13.0,22.0
4,19500.0,8.0,4.0,7.0,2015.0,55.0,13.0,22.0


In [28]:
final_col=list(final_col)
final_col.append('outcome')

In [29]:
final_train=train[final_col]
final_valid=valid[final_col]
final_test=valid[final_col]
train_model(final_train,final_valid,final_test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.024845 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 527
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 8
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501





wait a second to see accuracy on valid sets .. .
valid accuracy = 0.7475021559270776
wait another second to see accuracy on test set .. .
test accuracy = 0.7475021559270776


In [30]:
train_model(train,valid,test)

[LightGBM] [Info] Number of positive: 107340, number of negative: 193350
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.014086 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1553
[LightGBM] [Info] Number of data points in the train set: 300690, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.356979 -> initscore=-0.588501
[LightGBM] [Info] Start training from score -0.588501





wait a second to see accuracy on valid sets .. .
valid accuracy = 0.7466774415346332
wait another second to see accuracy on test set .. .
test accuracy = 0.746659055391359
